In [1]:
#!pip3 install .

In [2]:
import torch
from fastchat.conversation import Conversation, SeparatorStyle

In [3]:
from alpaca_lora_4bit.autograd_4bit import load_llama_model_4bit_low_ram, Autograd4bitQuantLinear, switch_backend_to
from safetensors.torch import load_file

switch_backend_to("cuda")

c:\Users\alex4321\anaconda3\envs\llama\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Triton not found. Please run "pip install triton".
Using CUDA implementation.


In [4]:
import alpaca_lora_4bit.matmul_utils_4bit
alpaca_lora_4bit.matmul_utils_4bit.act_order = False

In [5]:
model, tokenizer = load_llama_model_4bit_low_ram("vicuna-13b-GPTQ-4bit-128g/",
                                                 "vicuna-13b-GPTQ-4bit-128g/vicuna-13b-4bit-128g.safetensors",
                                                 groupsize=128)
model.half()
for n, m in model.named_modules():
    if isinstance(m, Autograd4bitQuantLinear):
        if m.is_v1_model:
            m.zeros = m.zeros.half()
        m.scales = m.scales.half()
        m.bias = m.bias.half()
from alpaca_lora_4bit.amp_wrapper import AMPWrapper
wrapper = AMPWrapper(model)
wrapper.apply_generate()

Loading Model ...


The safetensors archive passed at vicuna-13b-GPTQ-4bit-128g/vicuna-13b-4bit-128g.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.


Loaded the model in 6.61 seconds.


In [6]:
message = "What do you think the meaning of life is?"
conv = Conversation(
    name='vicuna_v1.1',
    system="A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.",
    roles=['USER', 'ASSISTANT'],
    messages=[],
    offset=0,
    sep_style=SeparatorStyle.ADD_COLON_TWO,
    sep=' ',
    sep2='</s>',
    stop_str=None,
    stop_token_ids=None
)
conv.append_message('USER', message)
conv.append_message('ASSISTANT', None)

In [7]:
prompt = conv.get_prompt()
batch = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
batch = {k: v.cuda() for k, v in batch.items()}

In [8]:
prompt_length = batch['input_ids'].shape[1]

In [9]:
for mode in ['disable', 'faster', 'old_faster']:
    alpaca_lora_4bit.matmul_utils_4bit.faster_mode = mode
    for i in range(5):
        with torch.no_grad():
            generated = model.generate(inputs=batch["input_ids"],
                                        do_sample=True, use_cache=True,
                                        repetition_penalty=1.1,
                                        max_new_tokens=25,
                                        temperature=0.1,
                                        top_p=0.97,
                                        top_k=40,
                                        return_dict_in_generate=True,
                                        output_attentions=False,
                                        output_hidden_states=False,
                                        output_scores=False)
        generated_text = tokenizer.decode(generated['sequences'][0][prompt_length:])
        print(mode, i, generated_text)

disable 0 As an AI language model, I don't have personal beliefs or opinions. However, the meaning of life is
disable 1 As an AI language model, I don't have personal beliefs or opinions, but I can provide some perspect
disable 2 As an AI language model, I don't have personal beliefs or opinions. However, the meaning of life is
disable 3 As an AI language model, I don't have personal beliefs or opinions, but

The post The Mean
disable 4 As an AI language model, I don't have personal beliefs or opinions. However, the meaning of life is
faster 0 As an AI language model, I don't have personal beliefs or opinions. However, the meaning of life is
faster 1 As an AI language model, I don't have personal beliefs or opinions. However, the meaning of life is
faster 2 As an AI language model, I don't have personal beliefs or opinions. However, the meaning of life is
faster 3 As an AI language model, I don't have personal beliefs or opinions. However, the meaning of life is
faster 4 As an AI langu